In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from gwpy.table import GravitySpyTable
from gwpy.timeseries import TimeSeries

# Get t0s

In [ ]:
# read GSpy table 
# https://zenodo.org/record/5649212/files/L1_O3a.csv, accessed on 17/07/2023
O3a = GravitySpyTable.read('L1_O3a.csv')

In [ ]:
# plot glitch params
columns = ['peak_time', 'duration', 'peak_frequency', 'central_freq', 'bandwidth', 'snr']

def plot_params(dframe, columns):
    for col in columns:
        plt.figure()
        plt.hist(dframe[col], 20)
        plt.title(col)
        plt.show()

In [ ]:
# remove duplicates
def rm_dupes(dframe):
    
    t0 = dframe['peak_time'] + dframe['peak_time_ns']/1e9
    t0 = np.array(t0)
    t0_shifted = np.roll(t0, -1)
    diff = t0_shifted - t0

    duplicate_idx = np.where(diff < 32)[0]
    
    cleaned_dframe = dframe.drop(dframe.index[duplicate_idx])

    return cleaned_dframe

In [ ]:
blips_lf = O3a[(O3a["ml_label"] == "Blip_Low_Frequency") 
            & (O3a["ml_confidence"] > 0.99) 
            & (O3a["snr"] > 8)  
            & (O3a["duration"] < 2)
            & (O3a["peak_frequency"] < 250)
            & (O3a["central_freq"] < 250)
            & (O3a["bandwidth"] < 250)     
            ]

blips_lf.sort('peak_time')
blips_lf = blips_lf.to_pandas()
blips_lf = rm_dupes(blips_lf)

len(blips_lf)

In [ ]:
plot_params(blips_lf, columns)

In [ ]:
# get t0
t0 = blips_lf['peak_time'] + blips_lf['peak_time_ns']/1e9
t0 = t0.to_numpy()
np.savetxt('t0_blips_lf.txt', t0)

# Get low frequency blips

In [ ]:
# files/dirs
fname = 't0_blips.txt'
fdir = 'data'

# params
asd_duration = 64
fsample = 512
glitch_duration = 2

# channel/frame data
frame = 'L1_HOFT_C01'
channel = 'L1:DCS-CALIB_STRAIN_CLEAN_C01'

# load data
t0s = np.loadtxt(fname)

# get glitches
for t0 in t0s:

    print(t0)

    gps_asd_pre = t0 - glitch_duration - asd_duration
    gps_asd_post = t0 + glitch_duration + asd_duration

    try:
        tseries_pre = TimeSeries.get(channel=channel, start=gps_asd_pre-asd_duration/2, end=gps_asd_pre+asd_duration/2, frametype=frame).resample(fsample)
        tseries_post = TimeSeries.get(channel=channel, start=gps_asd_post-asd_duration/2, end=gps_asd_post+asd_duration/2, frametype=frame).resample(fsample)

        asd_pre = tseries_pre.asd(4, 2, method='median')
        asd_post = tseries_post.asd(4, 2, method='median')

        asd_mean = (asd_pre + asd_post)/2

        tseries_glitch = TimeSeries.get(channel=channel, start=t0-glitch_duration*2, end=t0+glitch_duration*2, frametype=frame).resample(fsample)
        tseries_glitch_whitened = tseries_glitch.whiten(asd=asd_mean).crop(t0-glitch_duration, t0+glitch_duration)

        tseries_glitch_array = np.array([tseries_glitch.times, tseries_glitch]).T
        tseries_glitch_whitened_array = np.array([tseries_glitch_whitened.times, tseries_glitch_whitened]).T
        asd_mean_array = np.array([asd_mean.frequencies, asd_mean]).T

        np.save(f'{fdir}/{t0}_glitch_raw', tseries_glitch_array)
        np.save(f'{fdir}/{t0}_glitch_whitened', tseries_glitch_whitened_array)
        np.save(f'{fdir}/{t0}_asd_mean', asd_mean_array)

    except KeyboardInterrupt:
        print('KeyboardInterrupt')
        exit()

    except:
        print(f'Could not get the glitch {t0}')